<a href="https://colab.research.google.com/github/660710078/TopGun2025/blob/main/P'Nut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
from pathlib import Path
from types import SimpleNamespace

CONFIG_DIR = Path(__file__).resolve().parent / ".ultralytics"
CONFIG_DIR.mkdir(parents=True, exist_ok=True)
os.environ.setdefault("YOLO_CONFIG_DIR", str(CONFIG_DIR))

import cv2
import ultralytics as ul
from ultralytics import YOLO
from ultralytics.trackers.byte_tracker import BYTETracker
from ultralytics.utils import YAML


def load_tracker_config(name: str = "bytetrack") -> SimpleNamespace:
    cfg_path = Path(ul.__file__).resolve().parent / "cfg" / "trackers" / f"{name}.yaml"
    return SimpleNamespace(**YAML.load(cfg_path))


model = YOLO("yolov8n.pt")
CLASS_NAMES = model.names
cap = cv2.VideoCapture("drone2.mp4")
if not cap.isOpened():
    raise RuntimeError("Cannot open video")

frame_rate = cap.get(cv2.CAP_PROP_FPS) or 30
tracker = BYTETracker(args=load_tracker_config(), frame_rate=int(frame_rate))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    result = model(frame)[0]
    boxes = result.boxes
    if boxes is None:
        cv2.imshow("Tracking", frame)
        if cv2.waitKey(1) == 27:
            break
        continue

    tracks = tracker.update(boxes.cpu().numpy(), frame)
    frame_labels = []
    for x1, y1, x2, y2, track_id, score, cls_id, _ in tracks:
        x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
        track_id = int(track_id)
        cls_id = int(cls_id)
        label = CLASS_NAMES.get(cls_id, str(cls_id))
        frame_labels.append(label)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(
            frame,
            f"{label} ID:{track_id}",
            (x1, max(15, y1 - 5)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (0, 255, 0),
            2,
        )
    if frame_labels:
        unique_labels = ", ".join(sorted(set(frame_labels)))
        print(f"Frame {tracker.frame_id}: {unique_labels}")

    cv2.imshow("Tracking", frame)
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()


NameError: name '__file__' is not defined